In [1]:
%run ../../src/feature_selection_scripts/Feature_Selection_Bin_Cla_SRV.ipynb

['Seq', 'sTos', 'sTtl', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'AckDat', 'icmp', 'tcp', 'udp', 'ECO', 'FIN', 'INT', 'RST', 'Start']
(911917, 15)


In [2]:
import pickle
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = df[selected_feature_labels].values
X_train, X_test, y_binary_train, y_binary_test, y_attack_type_train, y_attack_type_test = train_test_split(X, y_binary, y_attack_type, test_size=0.25, random_state=42)
print(X_train.shape)


(911917, 15)


In [5]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y_attack_type_train_encoded = encoder.fit_transform(y_attack_type_train)
y_attack_type_train_encoded = to_categorical(y_attack_type_train_encoded)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

num_classes = y_attack_type_train_encoded.shape[1]
num_features = X_train.shape[1]
sequence_length = 1



In [7]:
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [8]:
model = Sequential([
    LSTM(50, input_shape=(1, X_train.shape[1])),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Attempt to train again
model.fit(X_train_reshaped, y_attack_type_train_encoded, epochs=20, batch_size=32)

Epoch 1/20


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 577us/step - accuracy: 0.9348 - loss: 0.2241
Epoch 2/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 585us/step - accuracy: 0.9892 - loss: 0.0402
Epoch 3/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 575us/step - accuracy: 0.9919 - loss: 0.0283
Epoch 4/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 561us/step - accuracy: 0.9937 - loss: 0.0224
Epoch 5/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 560us/step - accuracy: 0.9948 - loss: 0.0187
Epoch 6/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 571us/step - accuracy: 0.9953 - loss: 0.0166
Epoch 7/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 562us/step - accuracy: 0.9957 - loss: 0.0155
Epoch 8/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 552us/step - accuracy: 0.9960 - loss: 0.0139
Epoch 9/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 567us/step - accuracy: 0.9961 - loss: 0.0129
Epoch 10/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 561us/step - accuracy: 0.9963 - loss: 0.0127
Epoch 11/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 16s 550us/step - accuracy: 0.9966 -